<a href="https://colab.research.google.com/github/hmelberg/health-data-analytics/blob/develop/3_Expressions_Answering_hard_questions_with_an_easy_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Expressions: Answering hard questions with an easy query



Hans Olav Melberg, Nashville, November, 2019

## Introduction
In health analytics we often have to deal problems like the following:

* Identify all patients who received pharmaceutical X before Y

* How many patients who were first diagnosed with Ulcerative Colitis were later diagnosed with Crohn's Disease?

* Select all patients with diabetes who were admitted to hospital with a glocose level higher than 8 within 100 days of first being diagnosed with diabetes?
 

The aim of these questions can be to identify a particular group of patients that we want to study, or to search for indicators of problems and thereby improvements in treatment. In the diabetes example, patients who have been diagnosed should have received treatment so that they avoid ending up in the emergency room with high glucose levels. If the data shows that a sigificant share of diabetes patient have many episodes with high glucose levels, this is a sign that there is room for improvement.

This notebook is about different methods for how we can count or select patients based on a set of inclusion and exclusion criteria. We will start with a simple example of trying to identify patients who have received "A before B" and then end up with more complex expressions like "more than 5 events of A within 100 days after B." 


#Starting example: X before Y

Import libraries to be used




In [0]:

import pandas as pd
import numpy as np

Read synthetic medicare sample data on inpatient hospital stays

In [0]:
path = 'https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/Downloads/'
inpatient_file = 'DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.zip'

inpatient = pd.read_csv(path+inpatient_file)


Initial data preparation

In [0]:
# lower case column names
inpatient.columns=inpatient.columns.str.lower()

# easier to use pid than desynpuf_id
inpatient['pid']=inpatient.desynpuf_id

# useful to have pid in the index  
inpatient=inpatient.set_index('pid', drop=False)
inpatient.index.name='pid_index'

Look at the data

In [0]:
inpatient.head()

,desynpuf_id,clm_id,segment,clm_from_dt,clm_thru_dt,prvdr_num,clm_pmt_amt,nch_prmry_pyr_clm_pd_amt,at_physn_npi,op_physn_npi,ot_physn_npi,clm_admsn_dt,admtng_icd9_dgns_cd,clm_pass_thru_per_diem_amt,nch_bene_ip_ddctbl_amt,nch_bene_pta_coinsrnc_lblty_am,nch_bene_blood_ddctbl_lblty_am,clm_utlztn_day_cnt,nch_bene_dschrg_dt,clm_drg_cd,icd9_dgns_cd_1,icd9_dgns_cd_2,icd9_dgns_cd_3,icd9_dgns_cd_4,icd9_dgns_cd_5,icd9_dgns_cd_6,icd9_dgns_cd_7,icd9_dgns_cd_8,icd9_dgns_cd_9,icd9_dgns_cd_10,icd9_prcdr_cd_1,icd9_prcdr_cd_2,icd9_prcdr_cd_3,icd9_prcdr_cd_4,icd9_prcdr_cd_5,icd9_prcdr_cd_6,hcpcs_cd_1,hcpcs_cd_2,hcpcs_cd_3,hcpcs_cd_4,...,hcpcs_cd_7,hcpcs_cd_8,hcpcs_cd_9,hcpcs_cd_10,hcpcs_cd_11,hcpcs_cd_12,hcpcs_cd_13,hcpcs_cd_14,hcpcs_cd_15,hcpcs_cd_16,hcpcs_cd_17,hcpcs_cd_18,hcpcs_cd_19,hcpcs_cd_20,hcpcs_cd_21,hcpcs_cd_22,hcpcs_cd_23,hcpcs_cd_24,hcpcs_cd_25,hcpcs_cd_26,hcpcs_cd_27,hcpcs_cd_28,hcpcs_cd_29,hcpcs_cd_30,hcpcs_cd_31,hcpcs_cd_32,hcpcs_cd_33,hcpcs_cd_34,hcpcs_cd_35,hcpcs_cd_36,hcpcs_cd_37,hcpcs_cd_38,hcpcs_cd_39,hcpcs_cd_40,hcpcs_cd_41,hcpcs_cd_42,hcpcs_cd_43,hcpcs_cd_44,hcpcs_cd_45,pid
pid_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00013D2EFD8E45D1,00013D2EFD8E45D1,196661176988405,1,20100312.0,20100313.0,2600GD,4000.0,0.0,3.139084e+09,NaN,NaN,20100312,4580,0.0,1100.0,0.0,0.0,1.0,20100313,217,7802,78820,V4501,4280,2720,4019,V4502,73300,E9330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00013D2EFD8E45D1
00016F745862898F,00016F745862898F,196201177000368,1,20090412.0,20090418.0,3900MB,26000.0,0.0,6.476809e+09,NaN,NaN,20090412,7866,0.0,1068.0,0.0,0.0,6.0,20090418,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00016F745862898F
00016F745862898F,00016F745862898F,196661177015632,1,20090831.0,20090902.0,3900HM,5000.0,0.0,6.119985e+08,6.119985e+08,NaN,20090831,6186,0.0,1068.0,0.0,0.0,2.0,20090902,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,6186,V5866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00016F745862898F
00016F745862898F,00016F745862898F,196091176981058,1,20090917.0,20090920.0,3913XU,5000.0,0.0,4.971603e+09,NaN,1.119000e+09,20090917,29590,0.0,1068.0,0.0,0.0,3.0,20090920,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00016F745862898F
00016F745862898F,00016F745862898F,196261176983265,1,20100626.0,20100701.0,3900MB,16000.0,0.0,6.408400e+09,1.960860e+09,NaN,20100626,5849,0.0,1100.0,0.0,0.0,5.0,20100701,983,3569,4019,3542,V8801,78820,2639,7840,7856,4271,NaN,NaN,E8889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00016F745862898F


**Create a date column**

In order to answer questions about before and after, we need a column of dates. The *clm_from_dt* column has the date of the event in a numeric format ('20080301'). We convert the date to a string format using *astype(str)* and apply the function *pd.to_datetime()* to convert the string to a proper date format. We also tell the function how the string is formatted (year, month, day).


In [0]:
inpatient.clm_from_dt

pid_index
00013D2EFD8E45D1    20100312.0
00016F745862898F    20090412.0
00016F745862898F    20090831.0
00016F745862898F    20090917.0
00016F745862898F    20100626.0
                       ...    
FFF8B49458F9FFA7    20080329.0
FFF8B49458F9FFA7    20081012.0
FFFA950301FCA748    20100109.0
FFFC880E13CA6278    20081012.0
FFFE3ED9582AD46E    20090417.0
Name: clm_from_dt, Length: 66773, dtype: float64

In [0]:
# Convert to Int64 first to avoid problem of missing values
date_as_string = inpatient.clm_from_dt.astype('Int64').astype(str)

inpatient['date'] = pd.to_datetime(date_as_string, 
                                   format='%Y/%m/%d', 
                                   errors='coerce')
inpatient['date']

pid_index
00013D2EFD8E45D1   2010-03-12
00016F745862898F   2009-04-12
00016F745862898F   2009-08-31
00016F745862898F   2009-09-17
00016F745862898F   2010-06-26
                      ...    
FFF8B49458F9FFA7   2008-03-29
FFF8B49458F9FFA7   2008-10-12
FFFA950301FCA748   2010-01-09
FFFC880E13CA6278   2008-10-12
FFFE3ED9582AD46E   2009-04-17
Name: date, Length: 66773, dtype: datetime64[ns]

Having created a date column, we sort to make sure that the events are ordered by date for each individual

In [0]:
inpatient=inpatient.sort_values(['pid', 'date'])

We also need the codebook for the ICD 9 diagnosis:

In [0]:
icd9_codebook = pd.read_csv('https://www.nber.org/data/ICD9ProviderDiagnosticCodes/2010/icd9dx2010.csv')
icd9_codebook.head()

,dgns_cd,shortdesc,version,fyear
0,0010,Cholera d/t vib cholerae,27,2010
1,0011,Cholera d/t vib el tor,27,2010
2,0019,Cholera NOS,27,2010
3,0020,Typhoid fever,27,2010
4,0021,Paratyphoid fever a,27,2010


In [0]:
icd2text = icd9_codebook.set_index('dgns_cd').shortdesc.to_dict()

In [0]:
# Testing that the codebook works
inpatient.icd9_dgns_cd_1.value_counts().rename(index=icd2text)

Pneumonia, organism NOS     2453
Rehabilitation proc NEC     1807
Crnry athrscl natve vssl    1675
Septicemia NOS              1648
Obs chr bronc w(ac) exac    1558
                            ... 
Monoclon paraproteinemia       1
Chr stomach ulc hem/perf       1
Obsessive-compulsive dis       1
Prim thrombocytopen NEC        1
Schizo NEC-subchr/exacer       1
Name: icd9_dgns_cd_1, Length: 2740, dtype: int64

### Solution 1 to "X before Y": Create a new dataframe with the dates for when each person was first observed with CD or UC

In [0]:
# Identify relevant codes
#crohn's disease: 555x
cd_codes = [code for code, name in icd2text.items() if code.startswith('555')]
#ulcerative colitis: 556x
uc_codes = [code for code, name in icd2text.items() if code.startswith('556')]


In [0]:
cd_codes

['5550', '5551', '5552', '5559']

In [0]:
uc_codes

['5560', '5561', '5562', '5563', '5564', '5565', '5566', '5568', '5569']

In [0]:
# make a list of all columns with diagnostic codes
icd_columns = ['icd9_dgns_cd_1', 'icd9_dgns_cd_2', 'icd9_dgns_cd_3', 'icd9_dgns_cd_4',
       'icd9_dgns_cd_5', 'icd9_dgns_cd_6', 'icd9_dgns_cd_7', 'icd9_dgns_cd_8',
       'icd9_dgns_cd_9', 'icd9_dgns_cd_10']

In [0]:
# select rows with the cd and uc codes
cd_rows=inpatient[icd_columns].isin(cd_codes).any(axis=1)
uc_rows=inpatient[icd_columns].isin(uc_codes).any(axis=1)

In [0]:
# find the first cd or uc event for each person 
# first = when the date is minimum in the relvant rows for that person
first_cd = inpatient[cd_rows].groupby('pid').date.min()
first_uc = inpatient[uc_rows].groupby('pid').date.min()

In [0]:
# make a dataframe with the dates when CD or UC was observed
# one row per person

diagnose_date = first_cd.to_frame(name='first_cd')

diagnose_date['first_uc']=first_uc


In [0]:
# See how many had UC before CD
(diagnose_date.first_uc < diagnose_date.first_cd).sum()


2

In [0]:
# See how many had CD before UC
(diagnose_date.first_cd < diagnose_date.first_uc).sum()

1

###Solution 2: Make the *count_person()* function accept expressions

The first solution works fine, but it is a takes a little time to code and it does not generalize well. If we had to do it in another dataset, with different codes, we would have to redo the analysis. Given that temporal and logical queries are common in health analytics, we might think that there must be a better way (At least one other person often tinks that).

The general approach is adapt the function we have made - *count_persons()* to accept expressions and not just codes as input. That would make it possible to just write something like:

count_persons('cd before uc')

Let's start by making a simple *count_person* function that can handle (only) expressions like "X before/after Y":

In [0]:
def count_person(df, expression, col, pid='pid', date='date'):
  """
  Counts for how many persons the expressions "X after Y' is true
  """
  # Split the expression in a left and right side
  left_code, word, right_code = expression.split()
  
  # Here we assume there is a single code ono the right hand and left hand side
  left_rows = (df[col] == left_code)
  right_rows = (df[col] == right_code)
  
  # Make a dataframe with all the relevant individuals (one row per pereson)
  individuals = df[left_rows].pid.unique()
  dfi = pd.DataFrame(index=individuals)
  
  # Find the first date with the code on the left and right hand side (for each person)
  first_left_date = df[left_rows].groupby(pid)[date].min()
  first_right_date = df[right_rows].groupby(pid)[date].min()
  
  # Insert the code into the dataframe
  dfi['left_date']=first_left_date
  dfi['right_date']=first_right_date

  # Make the comparison of whether the left date is before the right date, and count
  if word=='before':
      left_before_right = dfi.left_date<dfi.right_date
      n = left_before_right.sum()
  elif word =='after':
      left_after_right = dfi.left_date>dfi.right_date
      n = left_after_right.sum()
  return n


In [0]:
count_person(df=inpatient, expression = '5560 before 5550', col='icd9_dgns_cd_1')

0

In [0]:
# A version of the function that accepts multiple codes and columns
def count_person(df, expr, col, pid='pid', date='date'):
  """
  Counts for how many persons the expressions "X after Y' is true
  """
  if isinstance(col, str):
    col=[col]
    
  left_code, word, right_code = expr.split()

  #Now we allow more than one code on both sides (split on comma, no space between)
  left_code = left_code.split(',')
  right_code = right_code.split(',')

  # We also allow more columns for the codes
  left_rows = df[col].isin(left_code).any(axis=1)
  right_rows = df[col].isin(right_code).any(axis=1)
  
  individuals = df[left_rows].pid.unique()
  dfi = pd.DataFrame(index=individuals)
  
  first_left_date = df[left_rows].groupby(pid)[date].min()
  first_right_date = df[left_rows].groupby(pid)[date].min()
  
  dfi['left_date']=first_left_date
  dfi['right_date']=first_right_date

  if word=='before':
      left_before_right = dfi.left_date<dfi.right_date
      n = left_before_right.sum()
  elif word =='after':
      left_after_right = dfi.left_date>dfi.right_date
      n = left_after_right.sum()
  
  return n





In [0]:
count_person(df=inpatient, expr = '5560,5561,5562,5563,5569 before 5550,5553,5559', 
             col=icd_columns)

0

##A list of common expressions

Having created a function that can count statements using keywords like before and after, you may soon realize - or your users will no doubt make you aware - that you could generalize the approach and add other types of expressions. For instance, consider a researcher who wants to know how many patients  with diabetes that have more than 2 hospital events with diabetes within 100 days of the first diagnosis. 

Onnce again, we will do the code and then add a function that would make this as easy, and generalizable, as:

```
select_persons('min 2 diabetes_codes within 100 days after 1st diabetes_code')
```

The general approach to solve expressions like this, is to parse it into smaller statements, solve each isolated statement, and then put it back to evaluate the overall relational statement.

In our case we have a general statement: ```x within y```
where 
* x is "min two diabetes codes"
* y is "100 days after 1st diabetes code"

One solution would be the following: 
- Find the first diabetes event for each person
- For all the other diabetes event, calculate the difference (in days) from the first event (for that person)
- Keep only the events that are less than 100 days away from the first event
- See which individuals that have at least two diabetes events in this reduced dataset

But this is only one of many possible expressions. The question is whether it is feasible to built a general approach that will cover as many many use cases as possible. 

Let's start by listing a number of possible expressions (X and Y are here sharhand for medical codes, say pharmaceutical X and Y):

```
X and Y
X before Y
max 3 of X
1st of X after 5th of Y
X within 100 days of Y
X within 100 days of 1st Y
X within first 20 of Y
X after 100 days of 3rd Y
X within 3rd to 8th of Y
max 3 of glucose>10
X after glucose>10
```

In addition the the expressons themselves, it would be nice to have an easy way to ...
* **include different column names***. 
For instance, `X in icd and Y in atc` would identify people who at one point had X in the icd column and Y in the atc column. 
* **Include existing lists of codes** If we already  have a list of codes, we should be able to refer to the list by its name in the columns expression, For instance: `@cd_codes after @uc_codes`
where the alpha symbol indicates that we are referrring to an external list (This is the same convention that is used by the eval function in Pandas).
* **Handle nested expressions** Often se use multiple and nested criteria to select patients: `((X or Y) before (Q and R)) within 100 days of 1st of Z`
* **Allow shorthands for codes** Instead of having to list all codes for cd (5550, 5551, 5552 etc), only might be allowed be allowed to write: `55* before 56*`

##General implementation approach for expressions

Here are the general steps for analyzing any expresssion:
0. **Preprocess the expressions**
Before doing anything, we should clean the expression. For instance, expand shorthand codes to their full codes, import external lists of codes and other formatting issues that are necessary to make the functions work.

1. **Break the expression down into smaller units** 
Example:```X and Y``` has two atoms that can be analyzed separately, before being evaluated together. ```max 3 of X and min 2 of Y``` also has two atoms, but this time each atom has a modifier. 

While it sounds easy to 'break an expression down intos it smaller units,' this begs two questions. First: What is the smallest unit? And second, more practically, how can we break down a nested expression in a way that makes it possible to evaluate it?

## Problem: Breaking up expressions
A first stab at breaking up expressions, is to split on logical operators (and, or). Hence,
'min 2 X and max 3 Y' would become two seperate conditions that we first evaluate separately ('min 2 X' and 'max 3 Y'). Breaking it down we could give them names, like condition 1 (c1) and 2 (c2). We then have:
* Original expression: 'min 2 X and max 3 Y'
* Transformed expression 'c1 and c2' where c1='min 2 X' and c2='max 3 Y'

In the case of logical operators this works fine. Each condition can be evaluated separately and we can then evaluate the transformed expression. Even nested operations are no problem: 

`(min 2 X or Y) and (Q or max 3 Z)`

would become: 

`(c1 or c2) and (c3 or c4)`

when we split on logical operators but maintain the 
parenthesis and Pandas itself can evaluate (nested)logical conditions using *pd.eval()* or *df.eval()*.

The problem arises when we include temporal operators. `min 2 X before Y` cannot be broken down and analyzed separately like an and/or statement. Instead, it must be considered as a fundamental unit  itself. True, it is composed of smaller atoms and when we analyze temporal statements we will break it down further, but we cannot start by splitting on temporal operators the same way we start by splitting on logical operators.

It becomes even worse when we allow nested temporal (and logical) operators: `(X or Y) before (Q and R)` It would be foolish to start by splitting on the logical operators since they are part of another fundamental unit - the temporal condition. In short, we wold want the parsing of the expression to be:
* (X or Y) before (Q and R)
* c1 before c2

One possible solution and implenentation of all this is as follows:
1. Make a function that finds the first non-nested expression in a string. Non-nested simply means the part that is encosed by parenthesis and ther are no other parenthesis inside.
2. Use the function recursively on the expression to find the first non-nested expressions, store it in a dictionary, replace the expression with a symbol (i1, i2 etc) and store the current expression after replacement of the nested expression with its symbol.

Here is a specific example using a slightly more complicated expression:
`(X or Y) before (Q and (R or S))`
* Find the first non-nested expression: (X or Y). Store this as i1 and the overall expression is now: i1 before (Q and (R or S))
* Find the first nested expresion in replaced expression from the previous step: R or S. Store this as i2 and replace. The new expression is now: i1 before (Q and i2)
* The first nested expression is in the previous expression is: Q and i2. Store this as i3. The (final) expression is now: i1 before i3

We have now broken down the expression, including logical and temporal operators, into smaller statements that can be evaluated in steps before we evaluate the final and last expression (i1 before i3).
## Problem 2: How to do implement the evaluations?
Inpractice this is done by first evaluating the first non-nested expression we found. This expression is not necessarily a simple condition, and may require further parsing, but we know one important thing. We do not have to worry about it being nested, so we can feed it to functions even id these functions do not handle nested expressions! This simpplifies the functions required to parse and solve the expression. We only need to store the result from evaluating the expression (a row based boolean series) in the dataframe under the name i1 and go on to evaluate the second non-nested expression we found. The evaluation of the second expression may involve the result from the first expression, but that is not problem since we now have evaluated and stored the result. We then nest our way back to the last expression.

What kind of functions do we need to evaluate simple statements?

**Create a row mask** 
At its base, each atom has one or more codes or row indicators. Our general approach should first mark the rows that contain these codes (a row mask). If the atom contains a code, we will use one of our appraches for marking rows (*str.contains()*, *isin()*). But the atom does not only have to be a code. It could be anything that will create a row mask. For instance, if the atom contains the name of a variable and a condition, we create a row_mask based on this. For instance `age>25` can be used as an expressions and it will create a row-mask marking all rows where age is larger than 25 as True in this first step. 

Note that expressions in the end are evaluated at the level of the individual, not events. This is intentional and very much the whole point of the new funcions. Pandas already has good implementations of row based queries using the *query()* method. If we want to identify rows with a code for patients that are above an age at the same time they have a code X, we could just write:

´df.query('age>25 and X in icd')´

In contrast to a row-based query, we are implementing a person based query. The expression *X and Y* will search for persons who at any point in time have had code X and Y. This is differentfrom a row based query that will only find persons who have had X and Y at the same time (in the same event/row). 

It is important to note the person based focus since it affects the interpretation of compound statements like *'X and age>25'* In the expression language we are creating, this will come back as true for every person who have code X and an obervation with an age larger than 25, but it does not require that the person should have received X when she was more than 25 years old. If you want the latter, you already have a good option using *query()*, and expressions should only be used when the focus is individuals and not rows. (One might extend the expression langauge to include row based selection like this: `X[age>25]`)

Although the final expression is evaluated at the person level, the starting point of the implementation is a row mask. Using this we can evaluate, for instance, whether a person has received a code (using *any()*). But before doing this, we must examine whether the row has a quantifier.

**Modify the row selection based on quantifiers**
In the previous step, all rows that satisfy a condition was marked as True (A condition is the existence of a code or something like age>25). Quantifiers may modify this result and produce a new row selection. This is easier than it sounds. For instance, having marked all rows with an IBD code, it is quite easy to group it by person id and selct only the first IBD event. The following quantifiers are easy to implement:
* Ordinal (1st, 2nd, 3rd etc)
* Interval (First 10, Last 5)
* Frequency (Min 2 of X)
And one might other quantifiers if needed (first 10 pct of X? Between 3rd and 8th of X)

It is easy, because Pandas has methods that allows you to take a boolean series and then create a cumulative sum for each person (groupby('pid').cumsum()). Moreover, these operations are order-preserving (i.e. the row order does not change). If the data is sorted by date, the result you get back from a groupby will have the same order as the original dataframe.




In [0]:
inpatient['cd_rows']=cd_rows
inpatient['uc_rows']=uc_rows


In [0]:
%%timeit
cd_rows.groupby(level=0).cumsum()

10 loops, best of 3: 28.2 ms per loop


In [0]:
%%timeit
inpatient.groupby('pid').cd_rows.cumsum()

10 loops, best of 3: 27.8 ms per loop


In [0]:
import numpy as np

In [0]:
cd_dates = np.where(cd_rows, inpatient.date, np.datetime64('NaT'))
uc_dates = np.where(uc_rows, inpatient.date, np.datetime64('NaT'))

In [0]:
cd_dates

array(['NaT', 'NaT', 'NaT', ..., 'NaT', 'NaT', 'NaT'],
      dtype='datetime64[ns]')

In [0]:
cd_bool = np.where(cd_rows, 1, np.nan)

In [0]:
inpatient['cd_bool'] = cd_bool

In [0]:
change_pid = inpatient.pid != inpatient.pid.shift(1)

In [0]:
inpatient['change_pid'] = change_pid

In [0]:
inpatient['change_pid']

pid_index
00013D2EFD8E45D1     True
00016F745862898F     True
00016F745862898F    False
00016F745862898F    False
00016F745862898F    False
                    ...  
FFF8B49458F9FFA7     True
FFF8B49458F9FFA7    False
FFFA950301FCA748     True
FFFC880E13CA6278     True
FFFE3ED9582AD46E     True
Name: change_pid, Length: 66773, dtype: bool

In [0]:
inpatient.loc[change_pid, 'cd_bool'] = 0

In [0]:
inpatient['cdonoff']=inpatient['cd_bool'].fillna(method='ffill')
inpatient['cdonoff']

pid_index
00013D2EFD8E45D1    0.0
00016F745862898F    0.0
00016F745862898F    0.0
00016F745862898F    0.0
00016F745862898F    0.0
                   ... 
FFF8B49458F9FFA7    0.0
FFF8B49458F9FFA7    0.0
FFFA950301FCA748    0.0
FFFC880E13CA6278    0.0
FFFE3ED9582AD46E    0.0
Name: cdonoff, Length: 66773, dtype: float64

In [0]:
inpatient[inpatient.pid.isin((cd_rows==1).index)].cdonoff.head(20)

pid_index
00013D2EFD8E45D1    0.0
00016F745862898F    0.0
00016F745862898F    0.0
00016F745862898F    0.0
00016F745862898F    0.0
00052705243EA128    0.0
0007F12A492FD25D    0.0
0007F12A492FD25D    0.0
0007F12A492FD25D    0.0
0007F12A492FD25D    0.0
000B97BA2314E971    0.0
000C7486B11E7030    0.0
00108066CA1FACCE    0.0
0011714C14B52EEB    0.0
0011CB1FE23E91AF    0.0
0011CB1FE23E91AF    0.0
00139C345A104F72    0.0
00139C345A104F72    0.0
0013E139F1F37264    0.0
00157F1570C74E09    0.0
Name: cdonoff, dtype: float64